In [1]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from slugify import slugify
import ast
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import booleanize
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

from scipy.stats import hmean
from collections import defaultdict, Counter, OrderedDict
from copy import deepcopy

import math
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import operator
import functools

import csv
import random 
import reorder_list


In [2]:
artists = pd.read_pickle("H.pkl")
playlists = pd.DataFrame.transpose(pd.read_pickle("W.pkl"))

artists = pd.DataFrame.divide(artists,np.max(artists,axis=1),axis=0)
playlists = pd.DataFrame.divide(playlists,np.max(playlists,axis=1),axis=0)

artists = artists ** (3/4)
playlists = playlists ** (2/3)


In [3]:
artists_df = pd.read_pickle("artists_lookup.pkl")
playlists_df = pd.read_pickle("playlists_lookup.pkl")

artists_df = artists_df.reset_index().set_index('name').loc[artists.columns].reset_index().set_index('id')
playlists_df = playlists_df.loc[playlists.columns]


In [4]:
tracks_df = pd.read_pickle("unique_tracks.pkl")


In [5]:
# tracks_df = pd.read_pickle("tracks_lookup.pkl")
# tracks_df["track_playlists"] = [ [] for _ in range(len(tracks_df)) ]

# for cur_playlist_slug, track_slug_list in playlists_df["playlist_tracks"].iteritems():
#     for cur_track_slug in track_slug_list:
#         try:
#             tracks_df["track_playlists"].loc[cur_track_slug].append(cur_playlist_slug)
#         except:
#             print(playlists_df.name.loc[cur_playlist_slug], cur_track_slug)


In [6]:
# work_tracks = tracks_df.copy()

# work_tracks["sort_name"] = work_tracks.name.str.lower().str.replace("\s", "")

# work_tracks["sort_track_artists"] = work_tracks["track_artists"].copy()

# map(lambda work_artists: work_artists.sort(), work_tracks["sort_track_artists"])

# work_tracks["sort_track_artists"] = list(map(tuple, work_tracks["sort_track_artists"]))

# work_tracks = work_tracks[work_tracks.duplicated(subset=["sort_name", "sort_track_artists"], keep=False)].sort_values(by="name")

# unique_keys = set(list(map(tuple,
#    work_tracks[["sort_name", "sort_track_artists"]].values.tolist()
# )))


In [7]:
# for cur_name, cur_artists in tqdm(unique_keys):
#     sub_tracks = tracks_df.loc[work_tracks[
#         ( work_tracks.sort_name == cur_name ) &
#         ( work_tracks.sort_track_artists == cur_artists )
#     ].index]
    
#     max_index = sub_tracks.track_playlists.map(lambda work_playlist: len(work_playlist)).idxmax()
#     new_playlists = list(set(functools.reduce(operator.concat, sub_tracks.track_playlists)))

#     for work_track in sub_tracks.itertuples():
#         work_track.track_playlists.clear()
        
#         if work_track.Index != max_index : continue
#         work_track.track_playlists.extend(new_playlists)
    
# tracks_df.to_pickle("unique_tracks.pkl")


In [8]:
component_names = artists.index.tolist()
assert component_names == playlists.index.tolist()

n_clusters = len(artists.index)
assert n_clusters == len(playlists.index)

n_artists = len(artists.columns)
n_playlists = len(playlists.columns)

print("clusters count:", n_clusters)
print("-------------------\n")

print("artists count:", n_artists)
print("playlists count:", n_playlists)


clusters count: 75
-------------------

artists count: 4803
playlists count: 8513


In [9]:
cur_map = {
    1: 74, 2: 60, 3: 59, 4: 46, 5: 72, 6: 75, 7: 36, 8: 67, 
    9: 48, 10: 73, 11: 65, 12: 11, 13: 57, 14: 43, 15: 52, 
    16: 34, 17: 17, 18: 69, 19: 30, 20: 71, 21: 54, 22: 68, 
    23: 70, 24: 23, 25: 47, 26: 64, 27: 58, 28: 66, 29: 32, 
    30: 22, 31: 44, 32: 7, 33: 61, 34: 33, 35: 38, 36: 50, 
    37: 51, 38: 45, 39: 14, 40: 6, 41: 35, 42: 8, 43: 3, 
    44: 26, 45: 5, 46: 62, 47: 13, 48: 41, 49: 18, 50: 53, 
    51: 28, 52: 27, 53: 37, 54: 63, 55: 39, 56: 9, 57: 4, 
    58: 24, 59: 25, 60: 31, 61: 21, 62: 42, 63: 55, 64: 15, 
    65: 49, 66: 56, 67: 40, 68: 1, 69: 29, 70: 2, 71: 20, 
    72: 10, 73: 12, 74: 16, 75: 19
}

label_dict = {
    1: "house (2605, daft punk)",
    2: "pop (1713, marshmello)",
    3: "soundtracks (1219, hans zimmer)",
    5: "downtempo (1160, air)",
    7: "electronic (1083, moby)",
    8: "cyberpunk (1051, perturbator)",
    12: "chill (931, tycho)",
    14: "abstract (887, chilly gonzales)",
    15: "geeky (858, rush)",
    16: "edm (844, tiesto)",
    17: "dance pop (841, beyonce)",
    18: "hard rock (837, metallica)",
    19: "indie (835, phoenix)",
    21: "hip hop (818, nas)",
    22: "chillhop (816, nujabes)",
    23: "metal (805, slipknot)",
    24: "modern rock (796, oasis)",
    30: "classic rock (709, beatles)",
    34: "90s rap (685, 2pac)",
    37: "ambient (611, album leaf)",
    39: "drum and bass (609, griz)",
    40: "pop punk (608, weezer)",
    42: "dubstep (567, skrillex)",
    47: "modern rap (538, drake)",
    51: "classical (491, mozart)",
    55: "chillwave (412, washed out)",
    61: "indietronica (358, the xx)",
    62: "nu jazz (345, zero 7)"
}

label_dict = dict( 
    (f"component_{cur_map[k]}", v) for (k, v) in label_dict.items() 
)


In [10]:
artists.rename(index=label_dict, inplace=True)
playlists.rename(index=label_dict, inplace=True)

drop_rows = [ component_name for component_name in component_names if component_name in artists.index ]

artists.drop(drop_rows, inplace=True)
playlists.drop(drop_rows, inplace=True)

assert artists.index.tolist() == playlists.index.tolist()


In [11]:
def flatten(cur_list):
    return [item for sublist in cur_list for item in sublist]


In [12]:
genre_dict = dict()

genre_list = artists.index.tolist()
assert genre_list == playlists.index.tolist()

artist_id_list = artists_df.index.tolist()

for cur_genre in genre_list: 
    print(cur_genre)
    
    artist_scalars = artists.loc[cur_genre]
    playlist_scalars = playlists.loc[cur_genre]
    
    cur_scores = []
    
    for cur_row in tqdm(tracks_df.itertuples(), total=len(tracks_df)):
        if len(cur_row.track_playlists) == 0 :
            cur_scores.append(0)
            continue
        
        artist_scalar_list = []
        
        for track_artist in cur_row.track_artists:
            if track_artist not in artist_id_list : continue   
            artist_scalar_list.append(
                artist_scalars[artists_df.name[track_artist]]
            )
            
        if len(artist_scalar_list) == 0:
            cur_scores.append(0)
            continue
            
        cur_score = np.max(artist_scalar_list)

        if cur_score < 1e-8 :
            cur_scores.append(0)
            continue
            
        followers_scalar = 1+np.log10(1+playlists_df.followers[cur_row.track_playlists])
        
        cur_score *= np.dot(
            followers_scalar, playlist_scalars[cur_row.track_playlists]
        )
        
        cur_scores.append(cur_score)

    genre_tracks = tracks_df.loc[
        [ tracks_df.index[i] for i in reversed(np.argsort(cur_scores)[-50000:]) ]
    ]
    
    top_track_list = []
    bot_track_list = []

    top_artists = []
    bot_artists = []
    
    track_times = []
    track_dict = defaultdict(int)

    for genre_track in genre_tracks.itertuples():
        if np.sum(track_times) >= ( 4 * 3600000 ) : break 

        if genre_track.duration_ms < 120000 : continue
        if genre_track.duration_ms > 720000 : continue

        is_over_limit = False
        for track_artist_id in genre_track.track_artists:
            if track_dict[track_artist_id] < 3 : continue
            is_over_limit = True

        if is_over_limit : continue
        for track_artist_id in genre_track.track_artists:
            track_dict[track_artist_id] += 1

        track_times.append(genre_track.duration_ms)

        if len(top_track_list) >= 4:
            bot_track_list.append(genre_track.Index)
            bot_artists.append(genre_track.track_artists)
            continue

        has_artist = False
        for track_artist_id in genre_track.track_artists:
            if track_artist_id in flatten(top_artists):
                has_artist = True
                break

        if has_artist:
            bot_track_list.append(genre_track.Index)
            bot_artists.append(genre_track.track_artists)
            continue

        top_track_list.append(genre_track.Index)
        top_artists.append(genre_track.track_artists)

    print(str(list(operator.concat(top_track_list, bot_track_list))[0:8]))
    genre_dict[cur_genre] = reorder_list.reorder_by(
        operator.concat(top_track_list, bot_track_list),
        operator.concat(top_artists, bot_artists),
    )
    print(str(list(genre_dict[cur_genre])[0:8]))


pop punk (608, weezer)



['7yCPwWs66K8Ba5lFuU2bcx', '6L89mwZXSOwYl76YXfX13s', '3Zwu2K0Qa5sT6teCCHPShP', '5wQnmLuC1W7ATsArWACrgW', '2TfSHkHiFO4gRztVIkggkE', '6nTiIhLmQ3FWhvrGafw2zj', '1oTo3ijRbaDAtrjJrGAPSw', '60a0Rd6pjrkxjPbaKzXjfq']
['7yCPwWs66K8Ba5lFuU2bcx', '6L89mwZXSOwYl76YXfX13s', '3Zwu2K0Qa5sT6teCCHPShP', '5wQnmLuC1W7ATsArWACrgW', '6nTiIhLmQ3FWhvrGafw2zj', '2TfSHkHiFO4gRztVIkggkE', '1oTo3ijRbaDAtrjJrGAPSw', '60a0Rd6pjrkxjPbaKzXjfq']
dubstep (567, skrillex)



['6VRhkROS2SZHGlp0pxndbJ', '0QIYINh2AwmOmdu8CRYvlw', '03EuhmMsEHVFI9ytainStU', '47r2YKldN36ykegWkciO0E', '5i7fZq3chLyCHo3VeB6goD', '4rwpZEcnalkuhPyGkEdhu0', '0u2AIKDVafHwCFQ9LDnqxH', '4A7qcJp2Sg867JST1pSPJz']
['6VRhkROS2SZHGlp0pxndbJ', '0QIYINh2AwmOmdu8CRYvlw', '03EuhmMsEHVFI9ytainStU', '47r2YKldN36ykegWkciO0E', '5i7fZq3chLyCHo3VeB6goD', '0u2AIKDVafHwCFQ9LDnqxH', '4A7qcJp2Sg867JST1pSPJz', '4rwpZEcnalkuhPyGkEdhu0']
chill (931, tycho)



['2qC1sUo8xxRRqYsaYEdDuZ', '2SPTGg9SC5MT1FwNX4IYfx', '2lJ4d8MCT6ZlDRHKJ1br14', '4IUgpL0CgSiloUOHzgd6Qe', '6koWevx9MqN6efQ6qreIbm', '7EE7jbv7Dv8ZkyWBlKhPXX', '6DsGpDGIDRv8EFGgW10T1p', '6WuZeRJ8RWPdNgtqLNqn4F']
['2qC1sUo8xxRRqYsaYEdDuZ', '2SPTGg9SC5MT1FwNX4IYfx', '2lJ4d8MCT6ZlDRHKJ1br14', '4IUgpL0CgSiloUOHzgd6Qe', '6koWevx9MqN6efQ6qreIbm', '6DsGpDGIDRv8EFGgW10T1p', '7Cg3F9ZsZ2TYUnlza49NYh', '7EE7jbv7Dv8ZkyWBlKhPXX']
modern rap (538, drake)



['2xLMifQCjDGFmkHkpNLD9h', '3KkXRkHbMCARz0aVfEt68P', '7KXjTSCq5nL1LoYtL7XAwS', '6DCZcSspjsKoFjzjrWoCdn', '0TlLq3lA83rQOYtrqBqSct', '2IRZnDFmlqMuOrYOLnZZyc', '7dt6x5M1jzdTEt8oCbisTK', '3swc6WTsr7rl9DqQKQA55C']
['2xLMifQCjDGFmkHkpNLD9h', '3KkXRkHbMCARz0aVfEt68P', '7KXjTSCq5nL1LoYtL7XAwS', '6DCZcSspjsKoFjzjrWoCdn', '0TlLq3lA83rQOYtrqBqSct', '2IRZnDFmlqMuOrYOLnZZyc', '7dt6x5M1jzdTEt8oCbisTK', '3swc6WTsr7rl9DqQKQA55C']
drum and bass (609, griz)


...


['0thLhIqWsqqycEqFONOyhu', '1TuopWDIuDi1553081zvuU', '6O7qFEXmLQcOsV37wrgJDz', '3WS7spXVlbeC5kjePmHMQW', '53qU2K55RJgfs9F50irpB3', '2qC1sUo8xxRRqYsaYEdDuZ', '0hNduWmlWmEmuwEFcYvRu1', '2SPTGg9SC5MT1FwNX4IYfx']
['0thLhIqWsqqycEqFONOyhu', '1TuopWDIuDi1553081zvuU', '6O7qFEXmLQcOsV37wrgJDz', '3WS7spXVlbeC5kjePmHMQW', '2qC1sUo8xxRRqYsaYEdDuZ', '53qU2K55RJgfs9F50irpB3', '0hNduWmlWmEmuwEFcYvRu1', '0kwBxSPK4vuQBfhlPGIutT']
hard rock (837, metallica)



['1hKdDCpiI9mqz1jVHRKG0E', '57bgtoPSgt236HzfBOd8kj', '0bVtevEgtDIeRjCJbK3Lmv', '1AhDOtG9vPSOmsWgNW0BEY', '08mG3Y1vljYA6bvDt4Wqkj', '2zYzyRzz6pRmhPzyfMEC8s', '3ZFwuJwUpIl0GeXsvF1ELf', '6NwbeybX6TDtXlpXvnUOZC']
['1hKdDCpiI9mqz1jVHRKG0E', '57bgtoPSgt236HzfBOd8kj', '0bVtevEgtDIeRjCJbK3Lmv', '1AhDOtG9vPSOmsWgNW0BEY', '08mG3Y1vljYA6bvDt4Wqkj', '3ZFwuJwUpIl0GeXsvF1ELf', '7o2CTH4ctstm8TNelqjb51', '2zYzyRzz6pRmhPzyfMEC8s']
metal (805, slipknot)



['2MvIMgtWyK88OiPi0J8Dg3', '2DlHlPMa4M17kufBvI2lEN', '40rvBMQizxkIqnjPdEWY1v', '3cwDSDzTiWr5H5xMQhQ6Mx', '61mWefnWQOLf90gepjOCb3', '6wqJeItl3Vc3az4ZicSQAB', '0snQkGI5qnAmohLE7jTsTn', '0EYOdF5FCkgOJJla8DI2Md']
['2MvIMgtWyK88OiPi0J8Dg3', '2DlHlPMa4M17kufBvI2lEN', '40rvBMQizxkIqnjPdEWY1v', '3cwDSDzTiWr5H5xMQhQ6Mx', '61mWefnWQOLf90gepjOCb3', '0snQkGI5qnAmohLE7jTsTn', '6W21LNLz9Sw7sUSNWMSHRu', '6wqJeItl3Vc3az4ZicSQAB']
downtempo (1160, air)



['5Nn2Dj7OQsGL6pgQ9iIzPp', '7IuxuJBP6kwOOu1ShFzmR1', '6tEaLXZlN8b71vWV1SSsRf', '5ZrrXIYTvjXPKVQMjqaumR', '0lnfcWRREvnGtXPP2ZDeq7', '40UlFC0dIxv7lAEJaPZLnm', '56oUD9ZzgSSDFgvLHxoWyZ', '2C1nOw1nRxJaBWb9I1x05A']
['5Nn2Dj7OQsGL6pgQ9iIzPp', '7IuxuJBP6kwOOu1ShFzmR1', '6tEaLXZlN8b71vWV1SSsRf', '5ZrrXIYTvjXPKVQMjqaumR', '0lnfcWRREvnGtXPP2ZDeq7', '40UlFC0dIxv7lAEJaPZLnm', '56oUD9ZzgSSDFgvLHxoWyZ', '2C1nOw1nRxJaBWb9I1x05A']
house (2605, daft punk)



['2pnJ87yTVpkgtZh6Tq4vKh', '6yCNHyzseat7CeTbeGBNaa', '5VWVVSABdgccboSdrKGiZH', '5ZrrXIYTvjXPKVQMjqaumR', '08DBDmvEgY2ZTVMVH2A8lt', '72MoCPAnz0UljAVmwHHw3Y', '6c9EGVj5CaOeoKd9ecMW1U', '4ua0IepBEISCWwF8dTJvcU']
['2pnJ87yTVpkgtZh6Tq4vKh', '6yCNHyzseat7CeTbeGBNaa', '5VWVVSABdgccboSdrKGiZH', '5ZrrXIYTvjXPKVQMjqaumR', '08DBDmvEgY2ZTVMVH2A8lt', '6c9EGVj5CaOeoKd9ecMW1U', '4ua0IepBEISCWwF8dTJvcU', '72MoCPAnz0UljAVmwHHw3Y']


In [14]:
with open('genres.csv', 'w', newline='') as csvfile:
    fieldnames = ['genre', 'tracks']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for genre_name, genre_tracks in genre_dict.items():
        writer.writerow({'genre': genre_name, 'tracks': genre_tracks})
    